# To calibration the polarizer and analyzer angle

In [20]:
from cProfile import label
from cmath import exp, pi
from numpy import tan, sin, sqrt,cos
from re import A
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as npy
from scipy.signal import savgol_filter
from matplotlib.pyplot import MultipleLocator
from iminuit import Minuit
import iminuit
from iminuit.cost import LeastSquares
# import latex
# plt.style.reload_library()
plt.style.use(['science'])
# plt.style.use(['science','ieee'])
# plt.style.use(['science','no-latex'])

## The dark current

In [15]:
def currentDark(filename):
    inputfile=open(filename,'r')
    measurement_dark=[]
    monitor_dark=[]
    while(1):
        try:
            temp=next(inputfile).split()
            if(temp[4]=='dark'):
                measurement_dark.append(1e6*float(temp[2]))
            elif(temp[4]=='dark1'):
                monitor_dark.append(1e6*float(temp[2]))
        except StopIteration:
            break
    dark_meansurement=npy.mean(npy.array(measurement_dark))
    dark_monitor=npy.mean(npy.array(monitor_dark))
    return dark_meansurement,dark_monitor
dark_measurement,dark_monitor = currentDark('measurement_dark.txt')

## Define class and fit function

In [16]:
def calculate_index(phi,delta):
    phi=float(phi*pi/180)
    delta=float(delta*pi/180)
    anglei=65/180*pi
    rho=tan(phi)*exp(delta*1j)
    n1=1.
    # print('rho',rho)
    epsilon= (n1**2) * (sin(anglei)**2) * ( 1+(tan(anglei)**2)*((1-rho)/(1+rho))**2) 
    ep_re=epsilon.real
    ep_im=epsilon.imag
    n2=(1./sqrt(2.) * sqrt( sqrt(ep_re**2+ep_im**2) +ep_re)).real
    k2=(1./sqrt(2.) * sqrt( sqrt(ep_re**2+ep_im**2) -ep_re)).real
    return n2,k2

def intensityFun(t,scale,a,b,w): #define fit function
    phi=w*t
    # return a0+a1*npy.cos(2*phi)
    return (1+a*npy.cos(2*phi)+ b*npy.sin(2*phi))*scale

class dataVar:
    def __init__(self,angleP):
        self.angleP=angleP
        self.current=[]
        self.time=[]
        self.angleA=[]

def reader(filename,angleP,use):
    data=dataVar(angleP)
    inputfile=open(filename,'r')
    i=0
    while(1):
        try:
            temp=next(inputfile).split()
            if(temp[4]!=('P'+str(angleP))):
                continue
            data.time.append(i*0.5)
            if(use=='measurement'):
                data.current.append(1e6*float(temp[2])-dark_measurement)
            elif(use=='monitor'):
                data.current.append(1e6*float(temp[2])-dark_monitor)
            i=i+1
        except StopIteration:
            break
    return data

## Calculate a series of $\alpha$ and $\beta$ for future fit

In [27]:
a=[3,0,3,3,0,2,1,1,1,2,2]  # the delay time of two Ammeters 
w0=0.035   # this is the estimated angular velocity
alpha=[]
alpha_err=[]
beta=[]
beta_err=[]
epsilon=[]
epsilon_err=[]
angleP=[]
angleP_radian=[]

for ip in range(11):
    angleA=[]
    angleP_temp=ip*9    #in degree
    angleP.append(angleP_temp)
    angleP_radian.append(angleP_temp/180*pi)
    window=10
    
    data_measurement=reader('measurement_caliAP.txt',angleP_temp,'measurement')
    data_monitor=reader('monitor_caliAP.txt',angleP_temp,'monitor')

    length=len(data_monitor.current)-20
    # print(length)
    b=0
    data_monitor.time=data_monitor.time[0:length]
    data_monitor.current=data_monitor.current[0+a[ip]:length+a[ip]]
    data_measurement.time=data_measurement.time[0:length]
    data_measurement.current=data_measurement.current[0+b:length+b]

    signal=npy.array(data_measurement.current)/npy.array(data_monitor.current)
    signal=savgol_filter(signal,window,3)  # Filtering operation
    # average of the end current
    mean_end=npy.mean(signal[720:])
    end_time=0
    for t in range(200):
        if(signal[680+t]>=mean_end):
            if(ip==10):
                end_time=680+t-6
            elif(ip==8):
                end_time=680+t-9
            else:
                end_time=680+t
            break
        else:
            continue
    
    signal_cut=npy.array(signal[0:end_time])
    signal_err=npy.sqrt(signal_cut)
    time_cut=npy.array(data_measurement.time[0:end_time])

    fig,ax=plt.subplots(2,1,sharex=True)
    ax[1].set_xlabel('Time (s)')
    ax[0].set_ylabel('Intensity')
    # ax[0].set_ylim()
    ax[0].plot(data_measurement.time,signal,label='Water 400nm',linewidth=1,color='orange')#,color='blue)
    ax[0].plot(time_cut,signal_cut,label='After cut',linestyle="--",color='k',linewidth=1)#,color='blue)
    ax[1].set_ylabel('Current (uA)')
    ax[1].plot(data_measurement.time,data_measurement.current,label='measurement')
    ax[1].plot(data_monitor.time,data_monitor.current,label='monitor')
    fig.suptitle(str(ip*9)+' Deg. Intensity Vs. Time')
    fig.subplots_adjust(hspace=0.1)
    fig.patch.set_facecolor('white')
    ax[0].legend()
    ax[1].legend()
    plt.close()
    fig.savefig('cali_figure/'+ str(ip*9)+ 'DegIntensityVsTime.png',dpi=1000)
    
    #time_cut and signal_cut ,signal_err is the x y yerr we need to fit.
    time_cut=time_cut-0.5*(len(time_cut)-1)
    least_squares=LeastSquares(time_cut,signal_cut,signal_err,intensityFun)
    m=Minuit(least_squares,scale=0,a=0,b=0,w=w0)
    m.limits[3]=(w0-0.005,w0+0.005)
    # m.limits[4]=(-0.3,0.3)
    m.migrad()
    m.hesse()
    para=m.values
    err=m.errors
    # p0=[0,0,0,w]
    # para,_=curve_fit(intensityFun,time_cut,signal_cut,p0)
    # print(para)
    # print(err)
    alpha.append(para[1])
    alpha_err.append(err[1])
    beta.append(para[2])
    beta_err.append(err[2])
    epsilon.append(1-para[1]**2-para[2]**2)
    epsilon_err.append(sqrt(4*para[1]**2*err[1]**2+4*para[2]**2*err[2]**2).real)
    signal_fit=intensityFun(time_cut,*para)
    ratio_fit=signal_cut/signal_fit
    fig,ax=plt.subplots(2,1,sharex=True)
    fit_info = [
        f"$\\chi^2$ / $n_\\mathrm{{dof}}$ = {m.fval:.1f} / {len(time_cut) - m.nfit}",
        ]
    for p, v, e in zip(m.parameters, m.values, m.errors):
        fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")
    ax[1].set_xlabel('Time (s)')
    ax[0].set_ylabel('Intensity')
    ax[0].plot(time_cut,signal_cut,label='Ori. signal',linewidth=1,color='orange')#,color='blue)
    ax[0].plot(time_cut,signal_fit,label='Fitted signal',linestyle="--",color='k',linewidth=1)#,color='blue)
    ax[1].set_ylabel('Ratio')
    ax[1].plot(time_cut,ratio_fit,'.',label='Ori. Vs. Fitted',markersize=1)
    plt.fill_between(time_cut,npy.mean(ratio_fit)+npy.std(ratio_fit),npy.mean(ratio_fit)-npy.std(ratio_fit),alpha=0.3)
    ax[1].fill
    fig.legend(title="\n".join(fit_info),bbox_to_anchor=(0.95, 0.7), loc=2, borderaxespad=0)
    fig.suptitle(str(ip*9)+' Deg. Fit Result')
    # ax[0].legend(title="\n".join(fit_info))
    # ax[0].legend()
    # ax[1].legend()
    fig.patch.set_facecolor('white')
    fig.savefig('cali_figure/Fit'+ str(ip*9)+ 'DegIntensityVsTime.png',dpi=1000)
    plt.close()
print(epsilon)
print(epsilon_err)

[0.14752463728910226, 0.14106802916035333, 0.13764203782043605, 0.1342042401531558, 0.13386644096961975, 0.13296937568651757, 0.13439032442626409, 0.140324981618561, 0.14476095014585222, 0.15508373354978178, 0.17764995720605742]
[0.25866989386384004, 0.23359838532647867, 0.18280641274323983, 0.18239025833971084, 1.0371769971862939, 0.9358298908368248, 1.4864221412205285, 0.046478256483071856, 0.026969298681801437, 0.21691050430527603, 0.3246821892380471]


## Fit the P and epsilon with angleP epsilon and epsilon_err

In [33]:
def residualFun(P,phi,delta,Ps,As,eta):
    As_radian=As/180*pi
    Ps_radian=Ps/180*pi
    phi_radian=phi/180*pi
    delta_radian=delta/180*pi
    a1=( ( tan(phi_radian) )**2-( tan(P-Ps_radian) )**2 )/( ( tan(phi_radian) )**2+( tan(P-Ps_radian) )**2 )
    b1=( 2*tan(phi_radian)*cos(delta_radian)*tan(P-Ps_radian) ) /( ( tan(phi_radian) )**2+( tan(P-Ps_radian) )**2 )
    a = 1/eta * ( a1*cos(2*As_radian)-b1*sin(2*As_radian) )
    b = 1/eta * ( a1*sin(2*As_radian)+b1*cos(2*As_radian) )
    return 1-a**2-b**2
def betaFun(P,phi,delta,Ps,As,eta):
    As_radian=As/180*pi
    Ps_radian=Ps/180*pi
    phi_radian=phi/180*pi
    delta_radian=delta/180*pi
    a1=( ( tan(phi_radian) )**2-( tan(P-Ps_radian) )**2 )/( ( tan(phi_radian) )**2+( tan(P-Ps_radian) )**2 )
    b1=( 2*tan(phi_radian)*cos(delta_radian)*tan(P-Ps_radian) ) /( ( tan(phi_radian) )**2+( tan(P-Ps_radian) )**2 )
    return  1/eta * ( a1*sin(2*As_radian)+b1*cos(2*As_radian) )

# least_squares_residual=LeastSquares(angleP_radian,epsilon,epsilon_err,residualFun)
least_squares_residual=LeastSquares(angleP_radian,epsilon,epsilon_err,betaFun)
m=Minuit(least_squares_residual,phi=0,delta=0,Ps=0,As=0,eta=1)
m.limits[2]=(0,180)
m.limits[3]=(0,180)
m.limits[4]=(0,1)
m.migrad()
m.hesse()
# epsilon_fit=residualFun(npy.array(angleP_radian),*m.values)
beta_fit=betaFun(npy.array(angleP_radian),*m.values)
fig,ax=plt.subplots()
fit_info = [
    f"$\\chi^2$ / $n_\\mathrm{{dof}}$ = {m.fval:.1f} / {len(time_cut) - m.nfit}",
    ]
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")
ax.set_xlabel('P (Deg.))')
ax.set_ylabel('$\\beta$')
# ax.errorbar(angleP,epsilon,epsilon_err,label='Ori.',linewidth=1,color='orange')#,color='blue)
ax.errorbar(angleP,beta,beta_err,label='Ori.',linewidth=1,color='orange')#,color='blue)
ax.plot(angleP,beta_fit,label='Fitted',linestyle="--",color='k',linewidth=1)#,color='blue)
# ax.plot(angleP,epsilon_fit,label='Fitted',linestyle="--",color='k',linewidth=1)#,color='blue)
# ax.set_ylabel('Ratio')
# ax.plot(time_cut,ratio_fit,'.',label='Ori. Vs. Fitted',markersize=1)
# plt.fill_between(time_cut,npy.mean(ratio_fit)+npy.std(ratio_fit),npy.mean(ratio_fit)-npy.std(ratio_fit),alpha=0.3)
# ax[1].fill
fig.legend(title="\n".join(fit_info),bbox_to_anchor=(0.95, 0.7), loc=2, borderaxespad=0)
fig.suptitle('Beta Fit Result')
# ax[0].legend(title="\n".join(fit_info))
# ax[0].legend()
# ax[1].legend()
fig.patch.set_facecolor('white')
fig.savefig('cali_figure/ResidualFit.png',dpi=1000)

plt.close()